In [2]:
import pandas as pd
import numpy as np


## Exercise 1

First we load and take a look at the dataset

In [28]:
dessert_data = pd.read_json("data/dessert.json")
dessert_data.sample(40)





,id,description,tags,manufacturer,type,portions,nutrients
345,18548,"ARCHWAY Home Style Cookies, Sugar",[],Archway Cookies,Cookies,"[{'amount': 1, 'unit': 'serving', 'grams': 24.0}]","[{'value': 4.88, 'units': 'g', 'description': ..."
78,18127,"Cake, snack cakes, creme-filled, chocolate wit...",[],None,Cake,"[{'amount': 1, 'unit': 'oz', 'grams': 28.35}, ...","[{'value': 3.63, 'units': 'g', 'description': ..."
382,18942,"Pie Crust, Cookie-type, Graham Cracker, Ready ...",[],None,Cake,"[{'amount': 1, 'unit': 'crust', 'grams': 183.0...","[{'value': 5.1, 'units': 'g', 'description': '..."
288,18459,"Breakfast tart, low fat",[],None,Cake,[],"[{'value': 3.99, 'units': 'g', 'description': ..."
168,18261,"English muffins, mixed-grain, toasted (include...",[],None,Savory Bread,"[{'amount': 1, 'unit': 'oz', 'grams': 28.35}, ...","[{'value': 9.9, 'units': 'g', 'description': '..."
67,18107,"Coffeecake, cinnamon with crumb topping, dry mix",[],,Cake,"[{'amount': 1, 'unit': 'oz', 'grams': 28.35}, ...","[{'value': 4.8, 'units': 'g', 'description': '..."
362,18630,"PILLSBURY, Chocolate Chip Cookies, refrigerate...",[],Pillsbury Co.,Cookies,"[{'amount': 1, 'unit': 'serving', 'grams': 28.0}]","[{'value': 3.5700000000000003, 'units': 'g', '..."
91,18147,Cheesecake commercially prepared,[],,Cake,"[{'amount': 1, 'unit': 'oz', 'grams': 28.35}, ...","[{'value': 5.5, 'units': 'g', 'description': '..."
361,18629,"PILLSBURY, Buttermilk Biscuits, Artificial Fla...",[],Pillsbury Co.,Biscuits,"[{'amount': 1, 'unit': 'serving', 'grams': 64.0}]","[{'value': 6.25, 'units': 'g', 'description': ..."
313,18507,KELLOGG'S Eggo Lowfat Blueberry Nutri-Grain Wa...,[],"Kellogg, Co.",Biscuits,"[{'amount': 1, 'unit': 'waffle, round (4 dia) ...","[{'value': 5.95, 'units': 'g', 'description': ..."


### Exercise 1a)

We decided to replace empty Strings and `None` values with the string "No manufacturer" for the **Manufacturer** column:

In [29]:
dessert_data.replace("", None, inplace=True)
dessert_data.fillna(value="no Manufacturer",inplace=True)
dessert_data["manufacturer"].sample(40) 

312            Kellogg, Co.
101         no Manufacturer
385         no Manufacturer
21          no Manufacturer
124         no Manufacturer
93          no Manufacturer
323         Archway Cookies
203         no Manufacturer
109         no Manufacturer
341         Archway Cookies
24          no Manufacturer
259         no Manufacturer
258         no Manufacturer
339         Archway Cookies
132         no Manufacturer
382         no Manufacturer
161         no Manufacturer
370      General Mills Inc.
219         no Manufacturer
317            Kellogg, Co.
359    The J.M. Smucker Co.
225         no Manufacturer
299            Kellogg, Co.
160         no Manufacturer
264         no Manufacturer
380         no Manufacturer
358    The J.M. Smucker Co.
274         no Manufacturer
338         Archway Cookies
131         no Manufacturer
17          no Manufacturer
7           no Manufacturer
337         Archway Cookies
56          no Manufacturer
201         no Manufacturer
91          no Manuf

Next, we need to extract the contents of the lists in the **tags** column. We define a function to extract the content of the list and return it:

In [52]:
def extract_list(my_list):
    if my_list.isempty():
        print("List empty!")
    else:
        count = 0
        for x in my_list:
            count += 1
            print(f"list value of element {}: {}" count, x)
        return my_list
dessert_data["tags"].applymap(extract_list)

SyntaxError: f-string: empty expression not allowed (2054961471.py, line 8)